In [40]:
from time import time
import matplotlib
import pandas as pd
import numpy as np
import re
import datetime
from time import time
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%matplotlib inline
matplotlib.style.use('ggplot')

In [41]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
campaign_data = pd.read_csv('campaign_data.csv')
# o_test = test.copy()

In [42]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0
3,44_231448,231448,44,05-09-2017 11:36,0,0
4,29_185580,185580,29,01-07-2017 18:01,0,0


In [43]:
test.head()

,id,campaign_id,user_id,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [44]:
tmp = train.groupby(['user_id']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['user_id', 'mean_clicks_user_id']
train = pd.merge(train, tmp, how='left', on=['user_id']) 
test = pd.merge(test, tmp, how='left', on=['user_id']) 

In [45]:
tmp = train.groupby(['campaign_id']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['campaign_id', 'mean_clicks_campaign_id']
train = pd.merge(train, tmp, how='left', on=['campaign_id']) 
test = pd.merge(test, tmp, how='left', on=['campaign_id']) 

In [46]:
campaign_data.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,29,Newsletter,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,http://r.newsletters.analyticsvidhya.com/7um44...
1,30,Upcoming Events,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,http://r.newsletters.analyticsvidhya.com/7up0e...
2,31,Conference,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,http://r.newsletters.analyticsvidhya.com/7usym...
3,32,Conference,24,19,7,1,\r\n \r\nHi ?\r\n \r\nBefore I dive into why y...,A.I. & Machine Learning: 5 reasons why you sho...,http://r.newsletters.analyticsvidhya.com/7uthl...
4,33,Others,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",http://r.newsletters.analyticsvidhya.com/7uvlg...


In [47]:
campaign_data.drop(['email_url'], axis=1, inplace=True)

In [48]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

In [49]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [50]:
# all text columns
text_cols = ['email_body', 'subject']
for col in text_cols:
    campaign_data[col+'_char'] = campaign_data[col].str.len()
    campaign_data[col+'_word'] = campaign_data[col].str.split().str.len()
    campaign_data[col+'avg_word'] = campaign_data[col].apply(lambda x: avg_word(x))
    campaign_data[col+'stopwords'] = campaign_data[col].apply(lambda x: len([x for x in x.split() if x in stop]))
    campaign_data[col+'numerics'] = campaign_data[col].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    campaign_data[col+'upper'] = campaign_data[col].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [51]:
cat_cols = ['communication_type']

In [52]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    le = LabelEncoder()
    le.fit(list(campaign_data[col]))
    campaign_data[col] = le.transform(list(campaign_data[col]))

In [53]:
campaign_data.head()

,campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_body_char,email_body_word,email_bodyavg_word,email_bodystopwords,email_bodynumerics,email_bodyupper,subject_char,subject_word,subjectavg_word,subjectstopwords,subjectnumerics,subjectupper
0,29,3,67,61,12,3,"Dear AVians,\r\n \r\nWe are shaping up a super...",Sneak Peek: A look at the emerging data scienc...,1498,243,5.037037,81,1,4,55,11,4.090909,2,0,1
1,30,5,18,14,7,1,"Dear AVians,\r\n \r\nAre your eager to know wh...",[July] Data Science Expert Meetups & Competiti...,204,34,4.794118,12,0,0,65,10,5.600000,1,0,0
2,31,0,15,13,5,1,Early Bird Pricing Till August 07  Save upto ...,Last chance to convince your boss before the E...,1085,183,4.726776,55,4,4,65,12,4.500000,4,0,0
3,32,0,24,19,7,1,\r\n \r\nHi ?\r\n \r\nBefore I dive into why y...,A.I. & Machine Learning: 5 reasons why you sho...,533,90,4.777778,32,2,1,77,13,5.000000,3,2,1
4,33,4,7,3,1,1,Fireside Chat with DJ Patil - the master is he...,"[Delhi NCR] Fireside Chat with DJ Patil, Forme...",739,118,4.983051,37,0,5,71,12,5.000000,1,0,3


In [54]:
campaign_data['external_links'] = campaign_data['total_links'] - campaign_data['no_of_internal_links']
campaign_data['link_image_ratio'] = campaign_data['total_links'] / campaign_data['no_of_images']
# campaign_data['links_per_section'] = campaign_data['total_links'] / campaign_data['no_of_sections']
# campaign_data['images_per_section'] = campaign_data['no_of_images'] / campaign_data['no_of_sections']

In [55]:
# creating a Series from campaign_data and test for text columns
body = pd.Series(campaign_data['email_body'].tolist()).astype(str)
sub = pd.Series(campaign_data['subject'].tolist()).astype(str)

In [56]:
# this function cleans punctuations, digits and irregular tabs. Then converts the sentences to lower
def sentence_clean(word):
    p1 = re.sub(pattern='(\W+)|(\d+)|(\s+)',repl=' ',string=word)
    p1 = p1.lower()
    return p1

In [57]:
# instantiate snowballstemmer
stemmer = SnowballStemmer(language='english')

# clean sentences
body = body.map(sentence_clean)
# split words
body = [[x for x in x.split()] for x in body]
# apply stemming
body = [[stemmer.stem(x) for x in x] for x in body]
# ignore words with less than Three characters
body = [[x for x in x if len(x) > 2] for x in body]
# join the words with space separation
body = [' '.join(x) for x in body]

In [58]:
# apply the above steps for name 
sub = sub.map(sentence_clean)
sub = [[x for x in x.split() ] for x in sub]
sub = [[stemmer.stem(x) for x in x] for x in sub]
sub = [[x for x in x if len(x) > 2] for x in sub]
sub = [' '.join(x) for x in sub]

In [59]:
# instantiate TfidfVectorizer for each of name, desc and keywords
tfidf_body = TfidfVectorizer(max_features=5, ngram_range=(1,3), stop_words='english')
tfidf_sub = TfidfVectorizer(max_features=3, ngram_range=(1,3), stop_words='english')

In [60]:
# fit_transform to each of the above dataframes
body = tfidf_body.fit_transform(body).todense()
sub = tfidf_sub.fit_transform(sub).todense()

In [61]:
# convert to dataframes
body = pd.DataFrame(body)
sub = pd.DataFrame(sub)
# rename variables
body.rename(columns= lambda x: 'variable_desc_'+ str(x), inplace=True)
sub.rename(columns= lambda x: 'variable_name_'+ str(x), inplace=True)

In [62]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [63]:
# top Ten words in desc
top_tfidf_feats(tfidf_body.idf_, tfidf_body.get_feature_names(), 10)

,feature,tfidf
0,hackathon,1.569095
1,learn,1.414944
2,scienc,1.256720
3,data scienc,1.256720
4,data,1.163629


In [64]:
# concat all the above Three dataframes and delete them once combined
combine = pd.concat([body, sub],axis=1)
del sub, body

In [65]:
# check the shape the combined dataframe
combine.shape

(52, 8)

In [66]:
campaign_data = pd.concat([campaign_data, combine],axis=1)
del combine

In [67]:
train = pd.merge(train, campaign_data, how='left', on=['campaign_id']) 
test = pd.merge(test, campaign_data, how='left', on=['campaign_id']) 
del campaign_data

In [68]:
train.drop(['is_open'], axis=1, inplace=True)

In [69]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [70]:
train['send_date'] = lookup(train['send_date'])
test['send_date'] = lookup(test['send_date'])

In [71]:
# date extraction
train['Year'] = train.send_date.dt.year
train['Month'] = train.send_date.dt.month
train['Day'] = train.send_date.dt.day
train['DayOfWeek'] = train.send_date.dt.dayofweek
train['WeekOfYear'] = train.send_date.dt.weekofyear
train['hour'] = train.send_date.dt.hour

In [72]:
# date extraction
test['Year'] = test.send_date.dt.year
test['Month'] = test.send_date.dt.month
test['Day'] = test.send_date.dt.day
test['DayOfWeek'] = test.send_date.dt.dayofweek
test['WeekOfYear'] = test.send_date.dt.weekofyear
test['hour'] = test.send_date.dt.hour

In [73]:
train.head()

,id,user_id,campaign_id,send_date,is_click,mean_clicks_user_id,mean_clicks_campaign_id,communication_type,total_links,no_of_internal_links,...,variable_desc_4,variable_name_0,variable_name_1,variable_name_2,Year,Month,Day,DayOfWeek,WeekOfYear,hour
0,42_14051,14051,42,2017-01-09 19:55:00,0,0.0,0.012824,3,88,79,...,0.243698,0.000000,0.000000,1.000000,2017,1,9,0,2,19
1,52_134438,134438,52,2017-02-11 12:53:00,0,0.0,0.014192,3,67,62,...,0.278071,0.000000,0.000000,1.000000,2017,2,11,5,6,12
2,33_181789,181789,33,2017-07-24 15:15:00,0,0.0,0.011449,4,7,3,...,0.429167,1.000000,0.000000,0.000000,2017,7,24,0,30,15
3,44_231448,231448,44,2017-05-09 11:36:00,0,0.0,0.011697,5,60,56,...,0.000000,0.518573,0.679827,0.518573,2017,5,9,1,19,11
4,29_185580,185580,29,2017-01-07 18:01:00,0,0.0,0.016549,3,67,61,...,0.583553,0.606494,0.795088,0.000000,2017,1,7,5,1,18


In [74]:
tmp = train.groupby(['Month']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['Month', 'mean_clicks_Month']
train = pd.merge(train, tmp, how='left', on=['Month']) 
test = pd.merge(test, tmp, how='left', on=['Month']) 

In [75]:
tmp = train.groupby(['Day']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['Day', 'mean_clicks_Day']
train = pd.merge(train, tmp, how='left', on=['Day']) 
test = pd.merge(test, tmp, how='left', on=['Day']) 

In [76]:
tmp = train.groupby(['DayOfWeek']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['DayOfWeek', 'mean_clicks_DayOfWeek']
train = pd.merge(train, tmp, how='left', on=['DayOfWeek']) 
test = pd.merge(test, tmp, how='left', on=['DayOfWeek']) 

In [77]:
tmp = train.groupby(['hour']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['hour', 'mean_clicks_hour']
train = pd.merge(train, tmp, how='left', on=['hour']) 
test = pd.merge(test, tmp, how='left', on=['hour']) 

In [78]:
tmp = train.groupby(['WeekOfYear']). \
    agg({'is_click' : [np.mean]}).reset_index()
tmp.columns = ['WeekOfYear', 'mean_clicks_WeekOfYear']
train = pd.merge(train, tmp, how='left', on=['WeekOfYear']) 
test = pd.merge(test, tmp, how='left', on=['WeekOfYear']) 

In [79]:
train.is_click.value_counts()

0    1010409
1      12782
Name: is_click, dtype: int64

In [80]:
train.head()

,id,user_id,campaign_id,send_date,is_click,mean_clicks_user_id,mean_clicks_campaign_id,communication_type,total_links,no_of_internal_links,...,Month,Day,DayOfWeek,WeekOfYear,hour,mean_clicks_Month,mean_clicks_Day,mean_clicks_DayOfWeek,mean_clicks_hour,mean_clicks_WeekOfYear
0,42_14051,14051,42,2017-01-09 19:55:00,0,0.0,0.012824,3,88,79,...,1,9,0,2,19,0.011480,0.014944,0.012131,0.011266,0.011444
1,52_134438,134438,52,2017-02-11 12:53:00,0,0.0,0.014192,3,67,62,...,2,11,5,6,12,0.014621,0.012101,0.014739,0.012553,0.014621
2,33_181789,181789,33,2017-07-24 15:15:00,0,0.0,0.011449,4,7,3,...,7,24,0,30,15,0.011739,0.011668,0.012131,0.013872,0.014076
3,44_231448,231448,44,2017-05-09 11:36:00,0,0.0,0.011697,5,60,56,...,5,9,1,19,11,0.011993,0.014944,0.014836,0.011697,0.011064
4,29_185580,185580,29,2017-01-07 18:01:00,0,0.0,0.016549,3,67,61,...,1,7,5,1,18,0.011480,0.014573,0.014739,0.020470,0.011536


In [81]:
# from sklearn.utils import resample
# # Separate majority and minority classes
# train_majority = train[train.is_click==0]
# train_minority = train[train.is_click==1]
 
# # Downsample majority class
# train_majority_downsampled = resample(train_majority, 
#                                  replace=False,    # sample without replacement
#                                  n_samples=12782,     # to match minority class
#                                  random_state=123) # reproducible results
 
# # Combine minority class with downsampled majority class
# train_downsampled = pd.concat([train_majority_downsampled, train_minority])
 
# # Display new class counts
# train_downsampled.is_click.value_counts()

In [82]:
target = train['is_click']
train.drop(['send_date', 'email_body', 'is_click', 'subject', 'id', 'user_id', 'campaign_id'], axis=1, inplace=True)
test_ids = test['id']
test.drop(['send_date', 'email_body', 'subject', 'id', 'user_id', 'campaign_id'], axis=1, inplace=True)

In [83]:
train.columns

Index(['mean_clicks_user_id', 'mean_clicks_campaign_id', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body_char', 'email_body_word', 'email_bodyavg_word',
       'email_bodystopwords', 'email_bodynumerics', 'email_bodyupper',
       'subject_char', 'subject_word', 'subjectavg_word', 'subjectstopwords',
       'subjectnumerics', 'subjectupper', 'external_links', 'link_image_ratio',
       'variable_desc_0', 'variable_desc_1', 'variable_desc_2',
       'variable_desc_3', 'variable_desc_4', 'variable_name_0',
       'variable_name_1', 'variable_name_2', 'Year', 'Month', 'Day',
       'DayOfWeek', 'WeekOfYear', 'hour', 'mean_clicks_Month',
       'mean_clicks_Day', 'mean_clicks_DayOfWeek', 'mean_clicks_hour',
       'mean_clicks_WeekOfYear'],
      dtype='object')

In [84]:
test.columns

Index(['mean_clicks_user_id', 'mean_clicks_campaign_id', 'communication_type',
       'total_links', 'no_of_internal_links', 'no_of_images', 'no_of_sections',
       'email_body_char', 'email_body_word', 'email_bodyavg_word',
       'email_bodystopwords', 'email_bodynumerics', 'email_bodyupper',
       'subject_char', 'subject_word', 'subjectavg_word', 'subjectstopwords',
       'subjectnumerics', 'subjectupper', 'external_links', 'link_image_ratio',
       'variable_desc_0', 'variable_desc_1', 'variable_desc_2',
       'variable_desc_3', 'variable_desc_4', 'variable_name_0',
       'variable_name_1', 'variable_name_2', 'Year', 'Month', 'Day',
       'DayOfWeek', 'WeekOfYear', 'hour', 'mean_clicks_Month',
       'mean_clicks_Day', 'mean_clicks_DayOfWeek', 'mean_clicks_hour',
       'mean_clicks_WeekOfYear'],
      dtype='object')

In [85]:
train.head()

,mean_clicks_user_id,mean_clicks_campaign_id,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body_char,email_body_word,email_bodyavg_word,...,Month,Day,DayOfWeek,WeekOfYear,hour,mean_clicks_Month,mean_clicks_Day,mean_clicks_DayOfWeek,mean_clicks_hour,mean_clicks_WeekOfYear
0,0.0,0.012824,3,88,79,13,4,1382,238,4.680672,...,1,9,0,2,19,0.011480,0.014944,0.012131,0.011266,0.011444
1,0.0,0.014192,3,67,62,10,4,971,166,4.710843,...,2,11,5,6,12,0.014621,0.012101,0.014739,0.012553,0.014621
2,0.0,0.011449,4,7,3,1,1,739,118,4.983051,...,7,24,0,30,15,0.011739,0.011668,0.012131,0.013872,0.014076
3,0.0,0.011697,5,60,56,19,6,273,43,5.162791,...,5,9,1,19,11,0.011993,0.014944,0.014836,0.011697,0.011064
4,0.0,0.016549,3,67,61,12,3,1498,243,5.037037,...,1,7,5,1,18,0.011480,0.014573,0.014739,0.020470,0.011536


In [86]:
test = test.reindex(columns=train.columns)

In [87]:
test.shape

(773858, 40)

In [88]:
test_ids.shape

(773858,)

In [89]:
categorical_features_indices = np.where(train.dtypes != np.float)[0]

In [90]:
# from imblearn.combine import SMOTETomek

# smt = SMOTETomek(ratio='auto')
# train_smote, target_smote = smt.fit_sample(train, target)

In [91]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [92]:
from sklearn.model_selection import train_test_split

# Split the 'features' and 'target' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train, 
                                                    target, 
                                                    test_size = 0.15, 
                                                    random_state = 77, 
                                                    stratify = target)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 869712 samples.
Testing set has 153479 samples.


In [93]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [94]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer , roc_auc_score, accuracy_score, confusion_matrix

def kFoldCV(clf, parameters, X_train, y_train, X_test, y_test, test): 
    scorer = make_scorer(roc_auc_score)
    grid_obj = GridSearchCV(clf, parameters, scoring=scorer, verbose=10, cv=2, n_jobs=-1)
    grid_fit = grid_obj.fit(X_train, y_train)
    best_clf = grid_fit.best_estimator_
    predictions = (clf.fit(X_train, y_train)).predict_proba(X_test)
    best_predictions = best_clf.predict_proba(X_test)
    best_predictions = [p[1] for p in best_predictions]
    print('best model: ', best_clf)
    print("Final roc_auc_score on the testing data: {:.4f}".format(roc_auc_score(y_test, best_predictions)))
    preds = best_clf.predict_proba(test)[:,1]
    return preds

In [95]:
def train_predict(clf, X_train, y_train, X_test, y_test, test):
    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_test)[:,1]
    print('{} has roc_auc_score: {}'.format(clf.__class__.__name__, roc_auc_score(y_test, preds)))
    preds = clf.predict_proba(test)[:,1]
    return preds

In [98]:
model1 = MLPClassifier(random_state=77, verbose=2)
preds3 = train_predict(model1, X_train, y_train, X_test, y_test, test)

Iteration 1, loss = 0.22426938
Iteration 2, loss = 0.10523045
Iteration 3, loss = 0.11315292
Iteration 4, loss = 0.09024885
Iteration 5, loss = 0.08050598
Iteration 6, loss = 0.06215939
Iteration 7, loss = 0.06154070
Iteration 8, loss = 0.05649696
Iteration 9, loss = 0.05608475
Iteration 10, loss = 0.05376302
Iteration 11, loss = 0.05094230
Iteration 12, loss = 0.05140093
Iteration 13, loss = 0.05007617
Iteration 14, loss = 0.04981141
Iteration 15, loss = 0.04894963
Iteration 16, loss = 0.04834578
Iteration 17, loss = 0.04838779
Iteration 18, loss = 0.04803869
Iteration 19, loss = 0.04762914
Iteration 20, loss = 0.04750595
Iteration 21, loss = 0.04739205
Iteration 22, loss = 0.04681894
Iteration 23, loss = 0.04659165
Iteration 24, loss = 0.04650712
Iteration 25, loss = 0.04615800
Iteration 26, loss = 0.04635830
Iteration 27, loss = 0.04619811
Iteration 28, loss = 0.04600543
Iteration 29, loss = 0.04597513
Iteration 30, loss = 0.04631670


C:\Users\shorake\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier has roc_auc_score: 0.9753590410502349


In [96]:
model1 = MLPClassifier(random_state=77)
preds3 = train_predict(model1, X_train, y_train, X_test, y_test, test)

model1 = LGBMClassifier(random_state=77,n_jobs=-1)
preds4 = train_predict(model1, X_train, y_train, X_test, y_test, test)

model1 = ExtraTreesClassifier(random_state=77, n_jobs=-1, n_estimators=100)
preds5 = train_predict(model1, X_train, y_train, X_test, y_test, test)

model1 = XGBClassifier(random_state=77, n_jobs=-1)
preds6 = train_predict(model1, X_train, y_train, X_test, y_test, test)

C:\Users\shorake\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier has roc_auc_score: 0.8948520558757785
LGBMClassifier has roc_auc_score: 0.9794103553635051
ExtraTreesClassifier has roc_auc_score: 0.9727742687438352
XGBClassifier has roc_auc_score: 0.9794911399310826


In [65]:
# params = {'n_estimators': [200], 'learning_rate': [1.0]}
# model1 = AdaBoostClassifier(random_state=77, )
# preds3 = kFoldCV(model1, params, X_train, y_train, X_test, y_test, test)

In [66]:
# params = {'learning_rate': [0.1]}
# model1 = LGBMClassifier(random_state=77,n_jobs=-1)
# preds4 = kFoldCV(model1, params, X_train, y_train, X_test, y_test, test)

In [67]:
# params = {'n_estimators': [100]}
# model1 = ExtraTreesClassifier(random_state=77, n_jobs=-1)
# preds5 = kFoldCV(model1, params, X_train, y_train, X_test, y_test, test)

In [68]:
# params = {}#{'colsample_bytree':[1, 0.9, 0.8], 'min_child_weight':[1, 1.2, 1.4]}
# model1 = XGBClassifier(random_state=77, n_jobs=-1)
# preds6 = kFoldCV(model1, params, X_train, y_train, X_test, y_test, test)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(random_seed=77, iterations=500, verbose= True, use_best_model=True, eval_metric='AUC')
model.fit(X_train, y_train, eval_set=(X_test, y_test))
preds_cb = model.predict_proba(test)[:,1]

0:	learn: 0.9701264	test: 0.9719227	best: 0.9719227 (0)	total: 1.12s	remaining: 9m 20s
1:	learn: 0.9781111	test: 0.9786469	best: 0.9786469 (1)	total: 2.17s	remaining: 8m 59s
2:	learn: 0.9781526	test: 0.9786939	best: 0.9786939 (2)	total: 3.1s	remaining: 8m 34s
3:	learn: 0.9781698	test: 0.9784996	best: 0.9786939 (2)	total: 4.1s	remaining: 8m 28s
4:	learn: 0.9782545	test: 0.9784704	best: 0.9786939 (2)	total: 5.04s	remaining: 8m 19s
5:	learn: 0.9782925	test: 0.9785032	best: 0.9786939 (2)	total: 6.03s	remaining: 8m 16s
6:	learn: 0.9784890	test: 0.9787918	best: 0.9787918 (6)	total: 7s	remaining: 8m 13s
7:	learn: 0.9785782	test: 0.9789581	best: 0.9789581 (7)	total: 7.96s	remaining: 8m 9s
8:	learn: 0.9785832	test: 0.9789599	best: 0.9789599 (8)	total: 8.9s	remaining: 8m 5s
9:	learn: 0.9786370	test: 0.9790038	best: 0.9790038 (9)	total: 9.89s	remaining: 8m 4s
10:	learn: 0.9786956	test: 0.9790100	best: 0.9790100 (10)	total: 10.9s	remaining: 8m 3s
11:	learn: 0.9786912	test: 0.9789954	best: 0.979010

93:	learn: 0.9793278	test: 0.9794485	best: 0.9794553 (90)	total: 1m 33s	remaining: 6m 42s
94:	learn: 0.9793321	test: 0.9794574	best: 0.9794574 (94)	total: 1m 34s	remaining: 6m 41s
95:	learn: 0.9793388	test: 0.9794608	best: 0.9794608 (95)	total: 1m 35s	remaining: 6m 40s
96:	learn: 0.9793451	test: 0.9794699	best: 0.9794699 (96)	total: 1m 36s	remaining: 6m 39s
97:	learn: 0.9793473	test: 0.9794758	best: 0.9794758 (97)	total: 1m 37s	remaining: 6m 38s
98:	learn: 0.9793484	test: 0.9794736	best: 0.9794758 (97)	total: 1m 38s	remaining: 6m 37s
99:	learn: 0.9793505	test: 0.9794728	best: 0.9794758 (97)	total: 1m 39s	remaining: 6m 36s
100:	learn: 0.9793519	test: 0.9794723	best: 0.9794758 (97)	total: 1m 40s	remaining: 6m 35s
101:	learn: 0.9793518	test: 0.9794719	best: 0.9794758 (97)	total: 1m 41s	remaining: 6m 34s
102:	learn: 0.9793539	test: 0.9794724	best: 0.9794758 (97)	total: 1m 42s	remaining: 6m 33s
103:	learn: 0.9793542	test: 0.9794722	best: 0.9794758 (97)	total: 1m 43s	remaining: 6m 32s
104:	l

183:	learn: 0.9794514	test: 0.9794968	best: 0.9795057 (139)	total: 3m 2s	remaining: 5m 13s
184:	learn: 0.9794526	test: 0.9794971	best: 0.9795057 (139)	total: 3m 3s	remaining: 5m 12s
185:	learn: 0.9794549	test: 0.9794951	best: 0.9795057 (139)	total: 3m 4s	remaining: 5m 11s
186:	learn: 0.9794553	test: 0.9794956	best: 0.9795057 (139)	total: 3m 5s	remaining: 5m 10s
187:	learn: 0.9794558	test: 0.9794960	best: 0.9795057 (139)	total: 3m 6s	remaining: 5m 9s
188:	learn: 0.9794569	test: 0.9794972	best: 0.9795057 (139)	total: 3m 7s	remaining: 5m 8s
189:	learn: 0.9794579	test: 0.9794983	best: 0.9795057 (139)	total: 3m 8s	remaining: 5m 7s
190:	learn: 0.9794589	test: 0.9794995	best: 0.9795057 (139)	total: 3m 9s	remaining: 5m 6s
191:	learn: 0.9794588	test: 0.9795024	best: 0.9795057 (139)	total: 3m 10s	remaining: 5m 5s
192:	learn: 0.9794593	test: 0.9795000	best: 0.9795057 (139)	total: 3m 11s	remaining: 5m 4s
193:	learn: 0.9794598	test: 0.9794990	best: 0.9795057 (139)	total: 3m 12s	remaining: 5m 3s
194

273:	learn: 0.9795332	test: 0.9794977	best: 0.9795057 (139)	total: 4m 31s	remaining: 3m 44s
274:	learn: 0.9795322	test: 0.9794941	best: 0.9795057 (139)	total: 4m 32s	remaining: 3m 43s
275:	learn: 0.9795332	test: 0.9794972	best: 0.9795057 (139)	total: 4m 33s	remaining: 3m 42s
276:	learn: 0.9795343	test: 0.9794988	best: 0.9795057 (139)	total: 4m 34s	remaining: 3m 41s
277:	learn: 0.9795354	test: 0.9795018	best: 0.9795057 (139)	total: 4m 35s	remaining: 3m 40s
278:	learn: 0.9795349	test: 0.9794999	best: 0.9795057 (139)	total: 4m 36s	remaining: 3m 39s
279:	learn: 0.9795368	test: 0.9794987	best: 0.9795057 (139)	total: 4m 37s	remaining: 3m 38s
280:	learn: 0.9795372	test: 0.9794981	best: 0.9795057 (139)	total: 4m 38s	remaining: 3m 37s
281:	learn: 0.9795372	test: 0.9794984	best: 0.9795057 (139)	total: 4m 39s	remaining: 3m 36s
282:	learn: 0.9795374	test: 0.9794927	best: 0.9795057 (139)	total: 4m 40s	remaining: 3m 35s
283:	learn: 0.9795406	test: 0.9794918	best: 0.9795057 (139)	total: 4m 41s	remain

363:	learn: 0.9796000	test: 0.9794952	best: 0.9795057 (139)	total: 6m 1s	remaining: 2m 14s
364:	learn: 0.9796003	test: 0.9794942	best: 0.9795057 (139)	total: 6m 2s	remaining: 2m 13s
365:	learn: 0.9796023	test: 0.9794976	best: 0.9795057 (139)	total: 6m 3s	remaining: 2m 13s
366:	learn: 0.9796055	test: 0.9794932	best: 0.9795057 (139)	total: 6m 4s	remaining: 2m 12s
367:	learn: 0.9796057	test: 0.9794931	best: 0.9795057 (139)	total: 6m 5s	remaining: 2m 11s
368:	learn: 0.9796061	test: 0.9794938	best: 0.9795057 (139)	total: 6m 6s	remaining: 2m 10s
369:	learn: 0.9796064	test: 0.9794931	best: 0.9795057 (139)	total: 6m 7s	remaining: 2m 9s
370:	learn: 0.9796064	test: 0.9794920	best: 0.9795057 (139)	total: 6m 8s	remaining: 2m 8s
371:	learn: 0.9796067	test: 0.9794894	best: 0.9795057 (139)	total: 6m 9s	remaining: 2m 7s
372:	learn: 0.9796075	test: 0.9794896	best: 0.9795057 (139)	total: 6m 10s	remaining: 2m 6s
373:	learn: 0.9796095	test: 0.9794882	best: 0.9795057 (139)	total: 6m 11s	remaining: 2m 5s
37

In [76]:
preds = (preds4+preds5+preds6+preds_cb)/4

In [77]:
sub = pd.DataFrame()
sub = pd.concat([test_ids, pd.Series(preds)], axis=1)
sub = sub.rename(columns={0:'is_click'})
sub[['is_click']] = sub[['is_click']].apply(pd.to_numeric)
sub.to_csv('a_submission.csv', index=False)

In [47]:
# from xgboost import XGBClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from lightgbm import LGBMClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression

# clf_C = LGBMClassifier(random_state=77,n_jobs=-1, verbose=10, learning_rate=0.01, n_estimators=100, max_depth=50)

# clf_C.fit(X_train, y_train)

# preds3 = clf_C.predict(X_test)

# print('{} has roc_auc_score: {}'.format(clf_C.__class__.__name__, roc_auc_score(y_test, preds3)))

LGBMClassifier has roc_auc_score: 0.5
